In [359]:
import math
import numpy as np
import matplotlib.pyplot as plt

In [360]:
class Value:
    def __init__(self, data, _children=(), _op='') -> None:
        self.value: int = data
        self.grad = .0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
    
    def __repr__(self) -> str:
        return f"Value(data={self.value}"
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other) # to make a+1.0 work
        out = Value(self.value + other.value, (self, other), "+")

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad

        out._backward = _backward
        return out 

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.value * other.value, (self, other), "*")
        def _backward():
            self.grad += other.value * out.grad
            other.grad += self.value * out.grad
        out._backward = _backward
        return out

    def __rmul__(self, other):
        return other * self
    
    def __radd__(self, other):
        return self + other
    
    def tanh(self):
        x = self.value
        t = (math.exp(2*x) - 1) / (math.exp(2*x) + 1)
        out = Value(t, (self,), "tanh")
        
        def _backward():
            self.grad = (1 - t**2) * out.grad

        out._backward = _backward
        return out
    
    def exp(self):
        x = self.value
        out = Value(math.exp(x), (self,))
        def _backward():
            self.grad += out.value * out.grad
        out._backward = _backward

        return out
    
    def __truediv__(self, other):
        return self * other**(-1)

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supports int/float powers"
        out = Value(self.value**other, (self,))

        def _backward():
            self.grad += other * self.value**(other-1) * out.grad
        out._backward = _backward
        return out
    
    def __sub__(self, other):
        return self + (-other)
    
    def __neg__(self):
        return self * -1
    
    def backward(self):
        # build the topological graph to order all of the children in the graph
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        # go one variable at a time and apply the chain rule to get its gradient
        self.grad = 1.0
        for v in reversed(topo):
            v._backward()

a = Value(32)
b = Value(2)
print(a, b)
print(a+b)
print(a*b)
d = a*b + a
print(d)

d.backward()

print(d.grad, a.grad, b.grad)

Value(data=32 Value(data=2
Value(data=34
Value(data=64
Value(data=96
1.0 3.0 32.0


In [361]:
print(d._prev, d._op)

{Value(data=64, Value(data=32} +


In [362]:
# Bug, should be 2 instead of 1, in the function _backward of addition,
#  we are overriding the gradient value (1), when self is the same object as other
#  when we use a variable more then ones
#  we should accumulate those gradients, multivariate case of chain rule
s = a + a
s.backward()
print(s.grad, a.grad)
# Expected 2, got 1 for s.grad

1.0 5.0


In [363]:
# Bug fixed
a = Value(32)
s = a + a
s.backward()
print(s.grad, a.grad)

1.0 2.0


To run back propagation, we need to store all the nodes in a topological order, then run the function in reverse order.

In [364]:
a = Value(2.0)
b = Value(4.0)
print(a/b)
a-b

Value(data=0.5


Value(data=-2.0

#### From scalars, gradients, neurons, and backpropagation To a MLP

In [365]:
class Neuron:
    def __init__(self, n) -> None:
        self.w = [Value(np.random.randn()) for _ in range(n)]
        self.b = Value(np.random.randn())
        
    def parameters(self):
        return self.w + [self.b]

    def __call__(self, x):
        a = sum((wi+xi for wi, xi in zip(self.w, x)), self.b)
        return a.tanh()
    
    
x = [2.0, 3.0]
n = Neuron(2)
n(x) # python will call n.__call__(x)

Value(data=0.9999989510163969

In [366]:
class Layer:
    def __init__(self, nin, nout) -> None:
        self.nin = nin
        self.nout = nout
        self.neurons = [Neuron(nin) for _ in range(nout)]
    
    def parameters(self):
        params = []
        for neuron in self.neurons:
            params.extend(neuron.parameters())
        return params
    
    def forward(self, x):
        return [neuron(x) for neuron in self.neurons]
    
    def __call__(self, x):
        return self.forward(x)[0] if self.nout == 1 else self.forward(x)
    
l = Layer(2, 3)
l(x)

[Value(data=0.9999925739334281,
 Value(data=0.999973652567766,
 Value(data=0.9999955858429052]

In [367]:
class MLP:
    def __init__(self, nin, nouts) -> None:
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
    
    def parameters(self):
        params = []
        for layer in self.layers:
            params.extend(layer.parameters())
        return params
    
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [368]:
input = [2.0, 3.0]
mlp = MLP(2, [3, 4, 1]) 
mlp(input)

Value(data=0.9948751482727038

In [369]:
# example
xs = [
    [2.0, 3.0, 1.0, 15.0],
    [1.0, 2.0, 3.0, 4.0],
    [3.0, 2.0, 1.0, 0.0]
]
target = [1.0, 0.0, 1.0]
pred = [mlp(xs[i]) for i in range(3)]
pred

[Value(data=0.9948751482727038,
 Value(data=0.9255864190976271,
 Value(data=0.9948751482727038]

let's define a loss to mesure the error of the model, we will use the mean squared error.

In [370]:
def mse(pred, target):
    return sum((p-t)**2 for p, t in zip(pred, target))

loss = mse(pred, target)
loss

Value(data=0.8567627474284216

In [371]:
loss.backward()

In [372]:
# check the gradients
mlp.layers[0].neurons[1].w[1].grad

0.003600700850319558

In [373]:
# weights and biases of all the network
mlp.parameters()

[Value(data=-1.1745984990765248,
 Value(data=-1.9079705594284473,
 Value(data=-1.063660778987389,
 Value(data=0.11912046977656406,
 Value(data=-1.6816276151023937,
 Value(data=1.3902314898828079,
 Value(data=-1.9071027640431564,
 Value(data=-0.1895946584929257,
 Value(data=-1.5856813359838235,
 Value(data=0.19827130943298815,
 Value(data=-0.32833234581224435,
 Value(data=0.7273317367482757,
 Value(data=-0.4288962787096917,
 Value(data=0.09582058896184041,
 Value(data=-0.7471910167898416,
 Value(data=-1.3686457254791715,
 Value(data=-2.020450802650164,
 Value(data=0.21318803384403726,
 Value(data=0.9364294046023246,
 Value(data=0.6412825808785294,
 Value(data=0.979869139612908,
 Value(data=-0.8162891143232834,
 Value(data=-1.75637302588548,
 Value(data=-0.8161966498560191,
 Value(data=-2.1703793279886114,
 Value(data=2.085807633793719,
 Value(data=0.11655715952430847,
 Value(data=-0.3431429078876648,
 Value(data=0.49221191638528705,
 Value(data=0.5360250534255223]

In [374]:
mlp.layers[0].neurons[1].w[1].value

-1.6816276151023937

In [375]:
lr = 0.01
for p in mlp.parameters():
    p.value -= lr * p.grad

# check the gradients
mlp.layers[0].neurons[1].w[1].value

-1.681663622110897

In [376]:
# forward pass, we expect the loss to decrease
pred = [mlp(xs[i]) for i in range(3)]
loss = mse(pred, target)
loss

Value(data=0.8495386963346085

The loss decreased. Yaaay!

Let's train it.

In [377]:
epochs = 100
lr = 0.01
for i in range(epochs):
    pred = [mlp(xs[i]) for i in range(3)]
    loss = mse(pred, target)
    for p in mlp.parameters():
        p.grad = .0

    loss.backward()
    for p in mlp.parameters():
        p.value -= lr * p.grad
    loss = mse(pred, target)

    # lr
    # if i % 100 == 0:
    #     lr *= 0.9 # decay the learning rate
    # this is a simple problem no need to decay lr

    print(f"Epoch: {i} -> Loss: {loss.value}")

pred = [mlp(xs[i]) for i in range(3)]
loss = mse(pred, target)
loss

Epoch: 0 -> Loss: 0.8495386963346085
Epoch: 1 -> Loss: 0.8416987366473648
Epoch: 2 -> Loss: 0.8331752616815645
Epoch: 3 -> Loss: 0.8238924426108042
Epoch: 4 -> Loss: 0.8137653515488731
Epoch: 5 -> Loss: 0.802699091415806
Epoch: 6 -> Loss: 0.7905879938301942
Epoch: 7 -> Loss: 0.7773149798241458
Epoch: 8 -> Loss: 0.762751225081122
Epoch: 9 -> Loss: 0.7467563352528163
Epoch: 10 -> Loss: 0.7291793210071186
Epoch: 11 -> Loss: 0.7098607683136523
Epoch: 12 -> Loss: 0.6886367240401775
Epoch: 13 -> Loss: 0.6653449477552339
Epoch: 14 -> Loss: 0.6398342882837845
Epoch: 15 -> Loss: 0.6119779714141635
Epoch: 16 -> Loss: 0.5816914386411618
Epoch: 17 -> Loss: 0.5489549190971685
Epoch: 18 -> Loss: 0.5138399855054666
Epoch: 19 -> Loss: 0.4765378136824522
Epoch: 20 -> Loss: 0.43738477325846337
Epoch: 21 -> Loss: 0.3968787229565336
Epoch: 22 -> Loss: 0.35567789153038315
Epoch: 23 -> Loss: 0.3145748777526356
Epoch: 24 -> Loss: 0.2744423718862616
Epoch: 25 -> Loss: 0.2361547589966109
Epoch: 26 -> Loss: 0.2

Value(data=0.004835307346619926

Wooooooow! It's working! Loss decreased by a lot!

In [378]:
pred

[Value(data=0.9511849156293668,
 Value(data=0.008335611698960703,
 Value(data=0.9511849156293668]

It's not perfect, but it's a good start.

Do not forget zero grad after each epoch, because the gradients are accumulated.

Let's verify the results with Pytorch

In [387]:
import torch

def test_addition():
    x = torch.tensor(2.0, requires_grad=True)
    y = torch.tensor(3.0, requires_grad=True)
    z = x + y
    z.backward()
    torch_grad_x = x.grad.item()
    torch_grad_y = y.grad.item()

    a = Value(2.0)
    b = Value(3.0)
    c = a + b
    c.backward()
    autodiff_grad_a = a.grad
    autodiff_grad_b = b.grad

    assert np.isclose(torch_grad_x, autodiff_grad_a), f"Gradients do not match for x: {torch_grad_x} vs {autodiff_grad_a}"
    assert np.isclose(torch_grad_y, autodiff_grad_b), f"Gradients do not match for y: {torch_grad_y} vs {autodiff_grad_b}"

    print("Addition tests pass")

def test_multiplication():
    x = torch.tensor(2.0, requires_grad=True)
    y = torch.tensor(3.0, requires_grad=True)
    z = x * y
    z.backward()
    torch_grad_x = x.grad.item()
    torch_grad_y = y.grad.item()

    a = Value(2.0)
    b = Value(3.0)
    c = a * b
    c.backward()
    autodiff_grad_a = a.grad
    autodiff_grad_b = b.grad

    assert np.isclose(torch_grad_x, autodiff_grad_a), f"Gradients do not match for x: {torch_grad_x} vs {autodiff_grad_a}"
    assert np.isclose(torch_grad_y, autodiff_grad_b), f"Gradients do not match for y: {torch_grad_y} vs {autodiff_grad_b}"

    print("Multiplication tests pass")

def test_tanh():
    x = torch.tensor(2.0, requires_grad=True)
    y = torch.tanh(x)
    y.backward()
    torch_grad_x = x.grad.item()

    a = Value(2.0)
    b = a.tanh()
    b.backward()
    autodiff_grad_a = a.grad

    assert np.isclose(torch_grad_x, autodiff_grad_a), f"Gradients do not match for x: {torch_grad_x} vs {autodiff_grad_a}"

    print("Tanh tests pass")

test_addition()
test_multiplication()
test_tanh()

Addition tests pass
Multiplication tests pass
Tanh tests pass
